In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import random
from sklearn.preprocessing import StandardScaler


In [75]:
final_df = pd.read_csv('../data/final_plot_emotion_20_genres.csv')
final_df 

,Name,Countries,Genres,Budget(USD)_Inflated,Domestic(USD)_Inflated,Domestic_Percentage,Foreign(USD)_Inflated,Foreign_Percentage,Worldwide(USD)_Inflated,Runtime(mins),Rating,Audience_Score,Critics_Score,plot_summary,Month,Day,Year,emotion,emotion_score
0,10 cloverfield lane,united states of america,"drama, horror, sci-fi, thriller",6.076746e+06,8.760602e+07,0.654010,4.634613e+07,0.345990,1.339521e+08,103,pg-13,0.79,0.91,"After surviving a car accident, Michelle (Mary...",Mar,11,2016,fear,0.679934
1,"10,000 bc",united states of america,"thriller, adventure, costume drama, action/adv...",1.441563e+08,1.301309e+08,0.351333,2.402606e+08,0.648667,3.703914e+08,109,pg-13,0.37,0.10,Mammoth hunter D'Leh (Steven Strait) has long ...,Mar,7,2008,fear,0.768002
2,12 strong,united states of america,"action, drama, history, war",3.979050e+07,5.209113e+07,0.644274,2.876133e+07,0.355726,8.085246e+07,130,r,0.62,0.50,"In the wake of Sept. 11, Capt. Mitch Nelson le...",Jan,19,2018,fear,0.908362
3,12 years a slave,united states of america,"biography, drama, history",2.539260e+07,7.195247e+07,0.301875,1.663993e+08,0.698125,2.383517e+08,134,r,0.90,0.95,"In the years before the Civil War, Solomon Nor...",Oct,18,2013,sadness,0.857475
4,127 hours,united states of america,"thriller, biographical film, drama, adventure",2.433688e+07,2.479013e+07,0.301870,5.733171e+07,0.698130,8.212184e+07,94,r,0.85,0.93,"While exploring a remote canyon in Utah, mount...",Jan,28,2011,fear,0.748359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,zookeeper,united states of america,"romance film, family film, comedy",1.065253e+08,1.070058e+08,0.470482,1.204330e+08,0.529518,2.274388e+08,102,pg,0.41,0.14,Kindhearted Griffin Keyes (Kevin James) is one...,Jul,8,2011,fear,0.889390
2233,zoolander 2,united states of america,"action, adventure, comedy, mystery, romance",6.076746e+07,3.506124e+07,0.508592,3.387664e+07,0.491408,6.893788e+07,101,pg-13,0.20,0.22,Former models Derek Zoolander (Ben Stiller) an...,Feb,12,2016,surprise,0.960526
2234,zoom,united states of america,"science fiction, action, fantasy, adventure",5.014200e+07,1.717625e+07,0.958658,7.407176e+05,0.041342,1.791697e+07,93,pg,0.34,0.05,"Capt. Zoom, or Jack (Tim Allen), as he is now ...",Aug,11,2006,neutral,0.680966
2235,zootopia,united states of america,"action, adventure, animation, comedy, crime, f...",1.823024e+08,4.147601e+08,0.333340,8.294952e+08,0.666660,1.244255e+09,108,pg,0.92,0.98,From the largest elephant to the smallest shre...,Mar,4,2016,neutral,0.720004


## preparing the data

In [76]:
# Dropping columns deemed irrelevant for the model
final_df = final_df.drop(columns=["Day","Name", "plot_summary", "Countries", "Worldwide(USD)_Inflated", "Domestic(USD)_Inflated", "Foreign(USD)_Inflated", "Worldwide(USD)_Inflated"])

In [77]:
final_df

,Genres,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,emotion_score
0,"drama, horror, sci-fi, thriller",6.076746e+06,0.654010,0.345990,103,pg-13,0.79,0.91,Mar,2016,fear,0.679934
1,"thriller, adventure, costume drama, action/adv...",1.441563e+08,0.351333,0.648667,109,pg-13,0.37,0.10,Mar,2008,fear,0.768002
2,"action, drama, history, war",3.979050e+07,0.644274,0.355726,130,r,0.62,0.50,Jan,2018,fear,0.908362
3,"biography, drama, history",2.539260e+07,0.301875,0.698125,134,r,0.90,0.95,Oct,2013,sadness,0.857475
4,"thriller, biographical film, drama, adventure",2.433688e+07,0.301870,0.698130,94,r,0.85,0.93,Jan,2011,fear,0.748359
...,...,...,...,...,...,...,...,...,...,...,...,...
2232,"romance film, family film, comedy",1.065253e+08,0.470482,0.529518,102,pg,0.41,0.14,Jul,2011,fear,0.889390
2233,"action, adventure, comedy, mystery, romance",6.076746e+07,0.508592,0.491408,101,pg-13,0.20,0.22,Feb,2016,surprise,0.960526
2234,"science fiction, action, fantasy, adventure",5.014200e+07,0.958658,0.041342,93,pg,0.34,0.05,Aug,2006,neutral,0.680966
2235,"action, adventure, animation, comedy, crime, f...",1.823024e+08,0.333340,0.666660,108,pg,0.92,0.98,Mar,2016,neutral,0.720004


In [78]:
# Standardizing 

columns_to_scale = ['Budget(USD)_Inflated', 'Domestic_Percentage', 'Foreign_Percentage', 'Runtime(mins)', 'Audience_Score', 'Critics_Score', 'emotion_score']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(final_df[columns_to_scale])
final_df[columns_to_scale] = scaled_data

final_df

,Genres,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,emotion_score
0,"drama, horror, sci-fi, thriller",-0.847910,0.477359,-0.477359,-0.315494,pg-13,1.051948,1.433935,Mar,2016,fear,0.048383
1,"thriller, adventure, costume drama, action/adv...",1.289405,-0.822013,0.822013,0.037533,pg-13,-1.195521,-1.530781,Mar,2008,fear,0.500661
2,"action, drama, history, war",-0.326059,0.435563,-0.435563,1.273128,r,0.142258,-0.066724,Jan,2018,fear,1.221487
3,"biography, drama, history",-0.548922,-1.034334,1.034334,1.508479,r,1.640570,1.580341,Oct,2013,sadness,0.960156
4,"thriller, biographical film, drama, adventure",-0.565264,-1.034355,1.034355,-0.845034,r,1.373015,1.507138,Jan,2011,fear,0.399786
...,...,...,...,...,...,...,...,...,...,...,...,...
2232,"romance film, family film, comedy",0.706919,-0.310516,0.310516,-0.374332,pg,-0.981476,-1.384375,Jul,2011,fear,1.124054
2233,"action, adventure, comedy, mystery, romance",-0.001360,-0.146912,0.146912,-0.433170,pg-13,-2.105211,-1.091564,Feb,2016,surprise,1.489377
2234,"science fiction, action, fantasy, adventure",-0.165830,1.785196,-1.785196,-0.903872,pg,-1.356055,-1.713788,Aug,2006,neutral,0.053685
2235,"action, adventure, animation, comedy, crime, f...",1.879862,-0.899257,0.899257,-0.021305,pg,1.747593,1.690146,Mar,2016,neutral,0.254166


In [79]:
# This part should already be in the main anaylsis!
# Filtering genres to only save top 20

# Assuming `final_df` is already loaded

# Split the genres into a list and explode the DataFrame
genres_exploded = final_df['Genres'].str.split(',').explode().str.strip()

# Count the occurrences of each genre
top_genres = genres_exploded.value_counts().head(20).index

# Filter the original DataFrame to only keep rows containing the top 20 genres
final_df['Filtered_Genres'] = final_df['Genres'].apply(
    lambda x: ', '.join([genre for genre in x.split(',') if genre.strip() in top_genres])
)

# Drop rows with no genres in the top 20
final_df = final_df[final_df['Filtered_Genres'].str.strip() != '']

# Optional: Drop the original Genres column and rename Filtered_Genres to Genres
final_df.drop(columns=['Genres'], inplace=True)
final_df.rename(columns={'Filtered_Genres': 'Genres'}, inplace=True)

# Print the updated DataFrame
final_df

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,emotion_score,Genres
0,-0.847910,0.477359,-0.477359,-0.315494,pg-13,1.051948,1.433935,Mar,2016,fear,0.048383,"drama, horror, sci-fi, thriller"
1,1.289405,-0.822013,0.822013,0.037533,pg-13,-1.195521,-1.530781,Mar,2008,fear,0.500661,"thriller, adventure, action/adventure, fant..."
2,-0.326059,0.435563,-0.435563,1.273128,r,0.142258,-0.066724,Jan,2018,fear,1.221487,"action, drama"
3,-0.548922,-1.034334,1.034334,1.508479,r,1.640570,1.580341,Oct,2013,sadness,0.960156,"biography, drama"
4,-0.565264,-1.034355,1.034355,-0.845034,r,1.373015,1.507138,Jan,2011,fear,0.399786,"thriller, drama, adventure"
...,...,...,...,...,...,...,...,...,...,...,...,...
2232,0.706919,-0.310516,0.310516,-0.374332,pg,-0.981476,-1.384375,Jul,2011,fear,1.124054,"romance film, family film, comedy"
2233,-0.001360,-0.146912,0.146912,-0.433170,pg-13,-2.105211,-1.091564,Feb,2016,surprise,1.489377,"action, adventure, comedy, mystery, romance"
2234,-0.165830,1.785196,-1.785196,-0.903872,pg,-1.356055,-1.713788,Aug,2006,neutral,0.053685,"science fiction, action, fantasy, adventure"
2235,1.879862,-0.899257,0.899257,-0.021305,pg,1.747593,1.690146,Mar,2016,neutral,0.254166,"action, adventure, comedy, crime, mystery"


In [80]:
# Splitting and one-hot encoding genres
genres_split = final_df['Genres'].str.get_dummies(sep=',')

# Combining one-hot encoded genres with final_df
df_ols = pd.concat([final_df, genres_split], axis=1)
df_ols = df_ols.drop(columns=['Genres'])


df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,drama,family film,fantasy,horror,indie,mystery,romance film,romantic comedy,science fiction,thriller
0,-0.847910,0.477359,-0.477359,-0.315494,pg-13,1.051948,1.433935,Mar,2016,fear,...,1,0,0,0,0,0,0,0,0,0
1,1.289405,-0.822013,0.822013,0.037533,pg-13,-1.195521,-1.530781,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,1
2,-0.326059,0.435563,-0.435563,1.273128,r,0.142258,-0.066724,Jan,2018,fear,...,0,0,0,0,0,0,0,0,0,0
3,-0.548922,-1.034334,1.034334,1.508479,r,1.640570,1.580341,Oct,2013,sadness,...,0,0,0,0,0,0,0,0,0,0
4,-0.565264,-1.034355,1.034355,-0.845034,r,1.373015,1.507138,Jan,2011,fear,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,0.706919,-0.310516,0.310516,-0.374332,pg,-0.981476,-1.384375,Jul,2011,fear,...,0,0,0,0,0,0,1,0,0,0
2233,-0.001360,-0.146912,0.146912,-0.433170,pg-13,-2.105211,-1.091564,Feb,2016,surprise,...,0,0,0,0,0,0,0,0,0,0
2234,-0.165830,1.785196,-1.785196,-0.903872,pg,-1.356055,-1.713788,Aug,2006,neutral,...,0,0,0,0,0,0,0,0,1,0
2235,1.879862,-0.899257,0.899257,-0.021305,pg,1.747593,1.690146,Mar,2016,neutral,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# one-hot encoding rating column, dropping the first column in order to avoid introducing multicollinearity
df_ols = pd.get_dummies(df_ols, columns=['Rating'], drop_first=True)
# one-hot encoding month column, dropping first column for same reason
df_ols = pd.get_dummies(df_ols, columns=['Month'], drop_first=True)
# same thing for the emotion column
df_ols = pd.get_dummies(df_ols, columns=['emotion'], drop_first=True)


In [82]:
df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Audience_Score,Critics_Score,Year,emotion_score,action,action/adventure,...,Month_May,Month_Nov,Month_Oct,Month_Sep,emotion_disgust,emotion_fear,emotion_joy,emotion_neutral,emotion_sadness,emotion_surprise
0,-0.847910,0.477359,-0.477359,-0.315494,1.051948,1.433935,2016,0.048383,0,0,...,False,False,False,False,False,True,False,False,False,False
1,1.289405,-0.822013,0.822013,0.037533,-1.195521,-1.530781,2008,0.500661,1,1,...,False,False,False,False,False,True,False,False,False,False
2,-0.326059,0.435563,-0.435563,1.273128,0.142258,-0.066724,2018,1.221487,0,0,...,False,False,False,False,False,True,False,False,False,False
3,-0.548922,-1.034334,1.034334,1.508479,1.640570,1.580341,2013,0.960156,0,0,...,False,False,True,False,False,False,False,False,True,False
4,-0.565264,-1.034355,1.034355,-0.845034,1.373015,1.507138,2011,0.399786,0,0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,0.706919,-0.310516,0.310516,-0.374332,-0.981476,-1.384375,2011,1.124054,0,0,...,False,False,False,False,False,True,False,False,False,False
2233,-0.001360,-0.146912,0.146912,-0.433170,-2.105211,-1.091564,2016,1.489377,0,0,...,False,False,False,False,False,False,False,False,False,True
2234,-0.165830,1.785196,-1.785196,-0.903872,-1.356055,-1.713788,2006,0.053685,1,0,...,False,False,False,False,False,False,False,True,False,False
2235,1.879862,-0.899257,0.899257,-0.021305,1.747593,1.690146,2016,0.254166,0,0,...,False,False,False,False,False,False,False,True,False,False


## Defining features and labels for the 4 different models:
Labels: Foreign/domestic percentage for prediction
Predictors: before and after 2010

In [83]:
# Movies before 2010
df_before_2010 = df_ols[df_ols['Year'] < 2010]

# Movies including and after 2010
df_2010_and_after = df_ols[df_ols['Year'] >= 2010]

### 1st model: Foreign Percentage prediction for years before 2010 (have to seperate years)

In [84]:
# Defining features and labels
y = df_ols["Foreign_Percentage"]

X = df_ols.drop(columns= ["Foreign_Percentage", 'Domestic_Percentage'])

# Adding a constant column for the OLS model
X = sm.add_constant(X)

In [85]:
# Training the OLS model
X = np.asarray(X, dtype=float)  # Ensure predictors are numeric
y = np.asarray(y, dtype=float)  # Ensure target is numeric
ols_model = sm.OLS(y, X).fit()

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     11.75
Date:                Thu, 19 Dec 2024   Prob (F-statistic):          5.12e-100
Time:                        22:49:54   Log-Likelihood:                -2837.1
No. Observations:                2237   AIC:                             5806.
Df Residuals:                    2171   BIC:                             6183.
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -53.3145      7.656     -6.964      0.0

### "2nd" model: Foreign Percentage prediction for years before 2010 (have to seperate years)

In [86]:
# Defining features and labels
y = df_ols["Domestic_Percentage"]

X = df_ols.drop(columns= ["Foreign_Percentage", 'Domestic_Percentage'])

# Adding a constant column for the OLS model
X = sm.add_constant(X)

In [87]:
# Training the OLS model
X = np.asarray(X, dtype=float)  # Ensure predictors are numeric
y = np.asarray(y, dtype=float)  # Ensure target is numeric
ols_model = sm.OLS(y, X).fit()

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     11.75
Date:                Thu, 19 Dec 2024   Prob (F-statistic):          5.12e-100
Time:                        22:49:59   Log-Likelihood:                -2837.1
No. Observations:                2237   AIC:                             5806.
Df Residuals:                    2171   BIC:                             6183.
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         53.3145      7.656      6.964      0.0